In [ ]:
import openslide
import os
import numpy as np

import torch
print(torch.cuda.is_available())

In [ ]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
from PIL import Image
import random
import openslide
import re
import math
from skimage.morphology import skeletonize
import xml.etree.ElementTree as ET
import tifffile
import skimage
from skimage import morphology
from mpl_toolkits.axes_grid1 import make_axes_locatable
import copy

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as trasnforms

import timm
import segmentation_models_pytorch as smp

import albumentations as A
from albumentations.pytorch import ToTensorV2
import h5py

In [ ]:
magnifications_40x = ["LN_068", "LN_069", "LN_070", "LN_071", "LN_072", "LN_073", "LN_074", "LN_075", "LN_076", "LN_077", 
"LN_078", "LN_079", "LN_080", "LN_081", "LN_082", "LN_083", "LN_084", "LN_085", "LN_086", "LN_087", 
"LN_088", "LN_089", "LN_090", "LN_091", "LN_092", "LN_093", "LN_094", "LN_095", "LN_096", "LN_097",
"LN_098",  "LN_099", "LN_100", "NOLN_067", "NOLN_068", "NOLN_069", "NOLN_070", "NOLN_071", "NOLN_072", "NOLN_073", "NOLN_074", 
"NOLN_075", "NOLN_076", "NOLN_077", "NOLN_078", "NOLN_079", "NOLN_080", "NOLN_081", "NOLN_082", 
"NOLN_083", "NOLN_084", "NOLN_085", "NOLN_086", "NOLN_087", "NOLN_088", "NOLN_089", "NOLN_090", 
"NOLN_091", "NOLN_092", "NOLN_093", "NOLN_094", "NOLN_095", "NOLN_096", "NOLN_097", "NOLN_098", 
"NOLN_099", "NOLN_100", "I_LN_001", "I_LN_002", "I_LN_003", "I_LN_004", 
    "I_LN_005", "I_LN_006", "I_LN_007", "I_LN_008", 
    "I_LN_009", "I_LN_010", "I_LN_011", "I_LN_012", "I_LN_013",
    "I_LN_014", "I_LN_015", "I_LN_016", "I_LN_017", "I_LN_018",
    "I_LN_019", "I_LN_020", "I_LN_021", "I_LN_022", "I_LN_023",
    "I_LN_024", "I_LN_025", "I_LN_026", "I_LN_027", "I_LN_028",
    "I_LN_029", "I_LN_030",
    "I_NOLN_001", "I_NOLN_002", "I_NOLN_003", "I_NOLN_004",
    "I_NOLN_005", "I_NOLN_006", "I_NOLN_007", "I_NOLN_008", 
    "I_NOLN_009", "I_NOLN_010", "I_NOLN_011", "I_NOLN_012",
    "I_NOLN_013", "I_NOLN_014", "I_NOLN_015", "I_NOLN_016",
    "I_NOLN_017", "I_NOLN_018", "I_NOLN_019", "I_NOLN_020", 
    "I_NOLN_021", "I_NOLN_022", "I_NOLN_023", "I_NOLN_024", 
    "I_NOLN_025", "I_NOLN_026", "I_NOLN_027", "I_NOLN_028", 
    "I_NOLN_029", "I_NOLN_030", "I_NOLN_031", "I_NOLN_032", 
    "I_NOLN_033", "I_NOLN_034", "I_NOLN_035", "I_NOLN_036", 
    "I_NOLN_037", "I_NOLN_038", "I_NOLN_039", "I_NOLN_040", 
    "I_NOLN_041", "I_NOLN_042", "I_NOLN_043", "I_NOLN_044", 
    "I_NOLN_045", "I_NOLN_046", "I_NOLN_047", "I_NOLN_048", 
    "I_NOLN_049", "I_NOLN_050", "I_NOLN_051", "I_NOLN_052", 
    "I_NOLN_053", "I_NOLN_054", "I_NOLN_055", "I_NOLN_056", 
    "I_NOLN_057", "I_NOLN_058", "I_NOLN_059", "I_NOLN_060", 
    "I_NOLN_061", "I_NOLN_062", "I_NOLN_063", "I_NOLN_064", 
    "I_NOLN_065", "I_NOLN_066", "I_NOLN_067", "I_NOLN_068", 
    "I_NOLN_069", "I_NOLN_070", "I_NOLN_071", "I_NOLN_072", 
    "I_NOLN_073"
    ]


In [ ]:
def random_seed(seed_value, use_cuda):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    random.seed(seed_value) 
    if use_cuda:
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars\n
        torch.backends.cudnn.deterministic = True  #needed\n
        torch.backends.cudnn.benchmark = False

In [ ]:
seed = 42
random_seed(seed,True)

In [ ]:
psize = (224,224)
overlap_3 = (192,192)
overlap_2 = (192,192)
overlap_1 = (156,156)
batch_size = 100

In [ ]:
model_path = {
    'level_3' : "Y:\\hsyoo\\pth\\pth_80\\level_3\\best_model_level3.pth",
    'level_2' : "Y:\\hsyoo\\pth\\pth_80\\level_2\\best_model_level2.pth",
    'level_1' : "Y:\\hsyoo\\pth\\pth_80\\level_1\\best_model_level1.pth"
}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
val_imtrans = A.Compose([
    A.Normalize(),
    ToTensorV2()
])

In [ ]:
def read_wsi_clf(slide, level, psize, overlap, net, x40): # wsi읽고 classification
    max_x = slide.level_dimensions[level][0]
    max_y = slide.level_dimensions[level][1]
    num = 0
    
    
    out_fill_3 = np.zeros(psize)
    out_fill_2 = np.zeros((psize[0]//2,psize[1]//2))
    out_fill_1 = np.zeros((psize[0]//4, psize[1]//4))
    if x40 == False:
        if level == 3:
            overlay = np.zeros([max_y, max_x])
            reference = np.zeros([max_y, max_x])
        elif level == 2:
            overlay = np.zeros([max_y//2, max_x//2])
            reference = np.zeros([max_y//2, max_x//2])
        elif level == 1:
            overlay = np.zeros([max_y//4, max_x//4])
            reference = np.zeros([max_y//4, max_x//4])
    elif x40 == True:
        if level == 4:
            overlay = np.zeros([max_y, max_x])
            reference = np.zeros([max_y, max_x])
        elif level == 3:
            overlay = np.zeros([max_y//2, max_x//2])
            reference = np.zeros([max_y//2, max_x//2])
        elif level == 2:
            overlay = np.zeros([max_y//4, max_x//4])
            reference = np.zeros([max_y//4, max_x//4])

    steps_x = int(math.ceil((max_x - overlap[0]) / float(psize[0] - overlap[0])))
    steps_y = int(math.ceil((max_y - overlap[1]) / float(psize[1] - overlap[1])))
    start_coords = list()
    
    for y in tqdm(range(0,steps_y)):
        col_list = list()
        
        for x in range(0, steps_x):
            x_start = x*(psize[0] - overlap[0])
            x_end = x_start + psize[0]
            y_start = y*(psize[1] - overlap[1])
            y_end = y_start + psize[1]
            
            if (x_end > max_x):
                x_overlap = x_end - max_x
                x_start = max_x - psize[0]
                x_end = max_x
            if(y_end > max_y):
                y_start = max_y - psize[1]
                y_end = max_y
                
            temp = np.array(slide.read_region((x_start*(2**(level)), y_start*(2**(level))), level, psize))#*************
            temp = temp[:,:,:3]
            temp = temp.astype(np.uint8)
            temp = val_imtrans(image = temp)['image'].unsqueeze(0).to(device).float() #스퀴즈(Squeeze) - 1인 차원을 제거한다.
            # ex) 어떤 텐서가 (3 × 1)의 크기를 가집니다. 두번째 차원이 1이므로 squeeze를 사용하면 (3,)의 크기를 가지는 텐서로 변경됩니다.
            
            if x40 == False:
                if level == 3:
                    start_coords.append((y_start,x_start))
                elif level == 2:
                    start_coords.append((y_start//2,x_start//2))
                elif level == 1:
                    start_coords.append((y_start//4,x_start//4))
            elif x40 == True:
                if level == 4:
                    start_coords.append((y_start,x_start))
                elif level == 3:
                    start_coords.append((y_start//2,x_start//2))
                elif level == 2:
                    start_coords.append((y_start//4,x_start//4))
            
            """
            batch size로 쌓기
            """
            if num==0:
                cat = temp
            else:
                cat = torch.cat((cat,temp), dim = 0)
            num+=1
            
            """
            model에 배치로 넣기
            """
            if num == batch_size:
                out = net(cat)
                out = F.softmax(out, dim = 1)
                out = out.detach().cpu().numpy()
                out = out[:,1]
                if x40 == False:
                    if level == 3:
                        for i in range(batch_size):
                            out_fill_3.fill(out[i]) #우선 np.zeros로 만들어 놓은 것 모델 출력 값으로 채움??
                            overlay[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] += out_fill_3
                            reference[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] +=1
                            if i ==batch_size-1:
                                num=0
                                start_coords = []
                    elif level == 2:
                        for i in range(batch_size):
                            out_fill_2.fill(out[i])
                            overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] += out_fill_2
                            reference[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] +=1
                            if i ==batch_size-1:
                                num=0
                                start_coords = []
                    elif level == 1:
                        for i in range(batch_size):
                            out_fill_1.fill(out[i])
                            overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] += out_fill_1
                            reference[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] +=1
                            if i ==batch_size-1:
                                num=0
                                start_coords = []
                elif x40 == True:
                    if level == 4:
                        for i in range(batch_size):
                            out_fill_3.fill(out[i])
                            overlay[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] += out_fill_3
                            reference[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] +=1
                            if i ==batch_size-1:
                                num=0
                                start_coords = []
                    elif level == 3:
                        for i in range(batch_size):
                            out_fill_2.fill(out[i])
                            overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] += out_fill_2
                            reference[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] +=1
                            if i ==batch_size-1:
                                num=0
                                start_coords = []
                    elif level == 2:
                        for i in range(batch_size):
                            out_fill_1.fill(out[i])
                            overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] += out_fill_1
                            reference[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] +=1
                            if i ==batch_size-1:
                                num=0
                                start_coords = []
            elif y==steps_y-1 and x==steps_x-1:
                out = net(cat)
                out = F.softmax(out, dim = 1)
                out = out.detach().cpu().numpy()
                out = out[:,1]
                if x40 == False:
                    if level == 3:
                        for i in range(num):
                            out_fill_3.fill(out[i])
                            overlay[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] += out_fill_3
                            reference[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] +=1
                    elif level ==2:
                        for i in range(num):
                            out_fill_2.fill(out[i])
                            overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] += out_fill_2
                            reference[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] +=1
                    elif level ==1:
                        for i in range(num):
                            out_fill_1.fill(out[i])
                            overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] += out_fill_1
                            reference[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] +=1
                elif x40 == True:
                    if level == 4:
                        for i in range(num):
                            out_fill_3.fill(out[i])
                            overlay[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] += out_fill_3
                            reference[start_coords[i][0]:start_coords[i][0]+psize[0], start_coords[i][1]:start_coords[i][1]+psize[1]] +=1
                    elif level ==3:
                        for i in range(num):
                            out_fill_2.fill(out[i])
                            overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] += out_fill_2
                            reference[start_coords[i][0]:start_coords[i][0]+psize[0]//2, start_coords[i][1]:start_coords[i][1]+psize[1]//2] +=1
                    elif level ==2:
                        for i in range(num):
                            out_fill_1.fill(out[i])
                            overlay[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] += out_fill_1
                            reference[start_coords[i][0]:start_coords[i][0]+psize[0]//4, start_coords[i][1]:start_coords[i][1]+psize[1]//4] +=1
    output = overlay / reference # overlay 가 무엇인지, reference 가 무엇인지
    print("overlay")
    print(overlay)
    print("\n")
    print("reference")
    print(reference)
    print("\n")
    print("output")
    print(output)
    print("\n")
    return output

In [ ]:
def make_white_image(png_save_dir, img, overlay, alpha):
    overlay[overlay >= 0.9] = 255
    overlay[overlay < 0.9] = 0
    overlay = overlay[:,:,np.newaxis]
    overlay = overlay.astype(np.uint8)
    blended = img * alpha + overlay * (1-alpha)
    blended = blended.astype(np.uint8)
    blended = Image.fromarray(blended)
    blended.save(os.path.join(png_save_dir, f"white\\{name}.png"))

In [ ]:
print("wsi_fns")
print(wsi_fns)
print('\n')
print("wsi_fns_iv")
print(wsi_fns_iv)
print('\n')
print("wsi_fns.extend(wsi_fns_iv)")
wsi_fns.extend(wsi_fns_iv)
print(wsi_fns)
print('\n')

print(f'wsi_fns_length:{length}')

In [ ]:
# 위 코드 테스트 및 수정


wsi_fns = list()
wsi_fns_iv = []
root_dir = "Y:\\data\\clam_data"
for patient in sorted(os.listdir(root_dir)):
    print(patient)
    if patient.split('.')[-1] == 'mrxs':
        print(os.path.join(root_dir, "data\\clam_data", patient))
        wsi_fns.append(os.path.join(root_dir, patient))
for patient in sorted(os.listdir("Y:\\VALIDATION")):
    if patient.split('.')[-1] == 'mrxs':
        wsi_fns_iv.append(os.path.join("Y:\\VALIDATION", patient))

print("wsi_fns")
print(wsi_fns)
print('\n')
print("wsi_fns_iv")
print(wsi_fns_iv)
print('\n')
print("wsi_fns.extend(wsi_fns_iv)")
wsi_fns.extend(wsi_fns_iv)
print(wsi_fns)
print('\n')

net_3 = timm.create_model("tf_efficientnet_b0_ns", pretrained = True, num_classes = 2)
net_3.load_state_dict(torch.load(model_path['level_3']))
net_3 = net_3.to(device).eval()

net_2 = timm.create_model("tf_efficientnet_b0_ns", pretrained = True, num_classes = 2)
net_2.load_state_dict(torch.load(model_path['level_2']))
net_2 = net_2.to(device).eval()

net_1 = timm.create_model("tf_efficientnet_b0_ns", pretrained = True, num_classes = 2)
net_1.load_state_dict(torch.load(model_path['level_1']))
net_1 = net_1.to(device).eval()

length = len(wsi_fns)
print(f'wsi_fns_length:{length}')

for idx, wsi_fn in enumerate(wsi_fns[104:length//2]):
    print("0000000000")
    print(wsi_fn)
    print("0000000000")
    name = wsi_fn.split("\\")[-1].split('.')[0]
    print(f"idx : {idx}, slide name : {name}")
    slide = openslide.OpenSlide(wsi_fn)

    if name in magnifications_40x:
        level_3 = 4
        level_2 = 3
        level_1 = 2
        x40 = True
    elif name not in magnifications_40x:
        level_3 = 3
        level_2 = 2
        level_1 = 1
        x40 = False

    if wsi_fn.split('\\')[2] == 'VALIDATION':
        mask_save_dir = "Y:\\mhson\\h5\\internal_valid\\ensemble"
        png_save_dir = "Y:\\mhson\\cancer_ensemble\\internal_valid"
    else:
        mask_save_dir = "Y:\\mhson\\h5\\train\\ensemble"
        png_save_dir = "Y:\\mhson\\cancer_ensemble\\train"

    img = np.array(slide.read_region((0,0), level_3, slide.level_dimensions[level_3]))[:,:,:3] #******************
    f = h5py.File(os.path.join(mask_save_dir, f"{name}.h5"), 'w')
    overlay_3 = read_wsi_clf(slide, level_3, psize, overlap_3, net_3, x40) #x40은 왜 주는건지 #read_wsi_clf의 출력 값은 원소값이 확률 값으로 이루어진 각 패치들이 담긴 리스트 ??
    f['level_3'] = overlay_3
    overlay_2 = read_wsi_clf(slide, level_2, psize, overlap_2, net_2, x40) #x40은 왜 주는건지
    f['level_2'] = overlay_2
    overlay_1 = read_wsi_clf(slide, level_1, psize, overlap_1, net_1, x40)
    f['level_1'] = overlay_1
    overlay = (overlay_3 + overlay_2 + overlay_1) / 3
    f['ensemble'] = overlay
    f.close()

    fig = plt.figure(dpi = 200)
    ax = fig.add_subplot(1,4,1)
    plt.imshow(img)
    plt.imshow(overlay_3, cmap = 'jet', alpha = 0.5)
    plt.axis('off')
    plt.title("2.5X") #25배

    ax = fig.add_subplot(1,4,2)
    plt.imshow(img)
    plt.imshow(overlay_2, cmap = 'jet', alpha = 0.5)
    plt.axis('off')
    plt.title("5X") #50배

    ax = fig.add_subplot(1,4,3)
    plt.imshow(img)
    plt.imshow(overlay_1, cmap = 'jet', alpha = 0.5)
    plt.axis('off')
    plt.title("10X") #100배

    ax = fig.add_subplot(1,4,4)
    plt.imshow(img)
    plt.imshow(overlay, cmap = 'jet', alpha = 0.5)
    plt.axis('off')
    plt.title("Ensemble") #level1,2,3 다 더해서 3으로 나눈 것
    plt.show()
    fig.savefig(os.path.join(png_save_dir, f"each\\{name}.png"))
    plt.close('all')
    plt.close(fig)

    make_white_image(png_save_dir, img, overlay, 0.5)

In [ ]:
p